In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
#/content/drive/My Drive/Infrrd.ai
cd /content/drive/My\ Drive/Infrrd.ai

/content/drive/My Drive/Infrrd.ai


In [4]:
ls

Final_Infrrd_ai.ipynb  prediction.csv  train.csv
Infrrd_ai.ipynb        test.csv        Untitled0.ipynb


In [5]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
# Imports
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
# Sklearn Imports
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb

In [8]:
# Keras Imports
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

Using TensorFlow backend.


In [50]:
# Checking the data before training the models
df_check = pd.read_csv("train.csv")
print(df_check.shape)
print(df_check.head(5))
print("*****")
print(df_check.tail(5))

(7200, 252)
   id   feature0   feature1  ...  feature248  feature249  label
0   0  25.208249 -16.745748  ...  130.068955   -8.231081      1
1   1 -86.931450   0.428227  ...  -26.164277   -4.909740      2
2   2  42.160934   7.857013  ...  120.510705    7.754377      0
3   3  20.666944   8.680642  ...  -40.426248    5.557970      1
4   4  35.946679   4.573736  ...  -23.468499  -11.269110      0

[5 rows x 252 columns]
*****
        id    feature0   feature1  ...  feature248  feature249  label
7195  7195  134.810289   4.051190  ...   29.500229    1.698117      4
7196  7196  124.446148   7.365961  ...   34.516724   18.481692      0
7197  7197  -29.478376  -4.853181  ...  -48.317597    1.550521      4
7198  7198   -7.350772 -13.460536  ...  -75.159849   -0.154655      3
7199  7199  -57.925764  -4.437577  ...  -69.663835   15.484705      1

[5 rows x 252 columns]


In [52]:
df_check.describe()

,id,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,feature25,feature26,feature27,feature28,feature29,feature30,feature31,feature32,feature33,feature34,feature35,feature36,feature37,feature38,...,feature211,feature212,feature213,feature214,feature215,feature216,feature217,feature218,feature219,feature220,feature221,feature222,feature223,feature224,feature225,feature226,feature227,feature228,feature229,feature230,feature231,feature232,feature233,feature234,feature235,feature236,feature237,feature238,feature239,feature240,feature241,feature242,feature243,feature244,feature245,feature246,feature247,feature248,feature249,label
count,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,...,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000
mean,3599.500000,2.504684,0.615413,3.016680,0.243273,-0.158600,0.506224,-0.276842,0.540687,0.608447,-0.216571,-0.640796,0.159247,-0.626838,0.242237,-0.119154,0.235442,-0.155578,-1.668230,-5.123835,2.364884,0.042340,-0.435367,0.541755,-1.373647,-0.547132,0.187925,-1.045961,-7.151048,0.547263,-0.597533,-0.151262,0.270463,0.165594,0.150636,-0.206447,0.189826,4.817891,0.390156,-0.325271,...,-0.663008,-0.211821,-0.134939,0.636004,-0.121269,-0.221302,-0.388627,0.263071,-0.130988,-0.328374,0.191363,0.016726,0.531364,0.180472,0.186527,-0.686691,0.648639,-0.254978,-0.331176,-0.292446,-0.049037,-0.294538,-0.656949,-0.360518,-0.557920,4.276076,0.197867,-0.154491,-0.582435,-0.160002,0.605181,-0.020437,-0.958585,-0.260203,-4.950015,-0.159411,0.672097,3.745077,0.273164,2.009167
std,2078.605302,67.755261,8.231712,65.188459,8.518782,8.190667,8.356110,8.365744,67.375739,8.133119,8.138978,8.355347,8.040079,8.091278,8.245793,8.340967,8.139238,8.221563,60.821216,68.689017,69.986011,8.236550,8.071465,8.014914,70.386622,8.165660,8.316625,8.136781,70.588599,8.187827,8.238298,8.098474,8.065563,8.015513,8.091156,8.398740,8.317210,69.011938,69.203657,7.995532,...,8.361792,8.081083,8.149971,8.315094,8.057111,8.544333,8.187305,8.262458,8.165704,7.903199,8.141047,8.250105,8.249824,8.369027,8.073964,8.155860,8.246036,8.323372,8.122141,8.188542,8.405445,8.041267,8.163116,8.222834,8.125606,62.531184,8.195790,8.422409,8.236291,8.239107,8.200987,8.390035,64.426649,8.162079,65.950715,8.218073,8.067559,69.372084,8.282975,1.411923
min,0.000000,-236.552276,-28.061843,-226.049557,-28.452153,-33.186906,-27.092009,-31.324360,-244.329485,-27.871326,-31.365588,-32.690045,-29.973536,-30.504274,-28.555918,-33.765700,-33.074933,-27.114766,-214.555481,-243.798895,-282.342354,-29.015075,-29.760087,-26.742305,-230.059512,-32.905233,-27.947689,-34.683619,-290.115260,-29.770034,-30.621127,-32.972128,-32.772882,-31.402202,-25.315229,-30.001914,-36.817586,-262.076661,-239.978368,-35.850329,...,-30.144300,-31.268302,-34.876253,-39.195365,-30.905763,-31.843946,-34.676757,-29.039543,-31.624231,-28.580088,-31.993894,-31.453285,-28.158385,-27.693307,-30.966224,-26.934303,-30.445080,-29.140377,-31.176162,-29.809755,-31.346080,

As we can observe from df_check.describe(),

-> We have 250 features(independent variables) and one Ground Truth.

-> Here I'm Dropping the "id" colum as it won't contribute in learning.

-> The dataset contains negative values to so I'm using "StandardScaler" for   "Normalizing" the input data to the models.

-> Also, there's no Categorical feature in our dataset.


In [54]:
# Lets check number of classes in our Ground Truth
y = df_check.iloc[:, df_check.shape[1]-1].values # Grabbing only the last column 
number_classes = np.unique(y).shape[0]
print("Number of unique classes in our dataset: ",number_classes)

Number of unique classes in our dataset:  5


In [57]:
# Lets check whether our dataset is balanced for all the classes
ret0, ret1, ret2, ret3, ret4 = [], [], [], [], []
i = 0
while i < y.shape[0]:
  if y[i] == 0: ret0.append(y[i])
  elif y[i] == 1: ret1.append(y[i])
  elif y[i] == 2: ret2.append(y[i])
  elif y[i] == 3: ret3.append(y[i])
  elif y[i] == 4: ret4.append(y[i])
  i += 1

print(i)
print("Class_0:",len(ret0),"Class_1:",len(ret1),"Class_2:",len(ret2),"Class_3:",
      len(ret3),"Class_4:",len(ret4)) 

7200
Class_0: 1419 Class_1: 1426 Class_2: 1485 Class_3: 1410 Class_4: 1460


The above observation concludes our dataset is well balanced for all the classes

In [0]:
np.random.seed(9) # To produce same result

I have experimented on Various models:

1) Logistic Regression

2) Support Vector Machine

3) XGBoost

4) K Nearest Neighbor

5) Artificial Neural Network with 2 hidden layers

In [0]:
# Function of all the Models I have experimented on
def Logistic_Regression_model(X_train, y_train,X_val,y_val):
  
  LR = LogisticRegression()
  parameters = {'penalty': ['l2'], 'C' : [0.001, 0.01, 0.1, 1, 10, 100, 1000], 
                'multi_class' :['multinomial'], 
                'solver': ['saga','lbfgs','newton-cg']}
  LR = GridSearchCV(LR, param_grid = parameters ,cv=5)
  LR.fit(X_train, y_train)
  score_train_acc = LR.score(X_train, y_train)
  print("Training Accuracy", score_train_acc)
  score_val_acc = LR.score(X_val, y_val)
  print("Validation Accuracy", score_val_acc) 
  y_train_check = LR.predict(X_train)
  print("**** TRAINING INFORMATION ****")
  print("Classification Report and Confusion Matrix on Training")
  print(classification_report(y_train, y_train_check))
  print(confusion_matrix(y_train, y_train_check, labels=np.unique(y_train)))
  print("------------------------------")
  print("**** VALIDATION INFORMATION ****")
  y_pred_LR = LR.predict(X_val)
  print("Classification Report and Confusion Matrix on Validation")
  print(classification_report(y_val, y_pred_LR))
  print(confusion_matrix(y_val, y_pred_LR, labels=np.unique(y_pred_LR)))
  pkl_filename = "LR_pickle_model.pkl"
  with open(pkl_filename, 'wb') as file:
    pickle.dump(LR, file)
  return score_train_acc, score_val_acc

def SVM_model(X_train, y_train,X_val,y_val):
  svm = SVC(random_state = 47, C = 0.1, kernel='rbf', 
                  class_weight = 'balanced', decision_function_shape = "ovr")
  parameters = {'C':[0.1, 0.001, 1],'decision_function_shape':['ovo', 'ovr']}
  svm = GridSearchCV(svm, param_grid= parameters ,cv=5)
  svm.fit(X_train, y_train)
  score_train_acc = svm.score(X_train, y_train)
  print("Training Accuracy", score_train_acc)
  score_val_acc = svm.score(X_val, y_val)
  print("Validation Accuracy", score_val_acc) 
  y_train_check = svm.predict(X_train)
  print("**** TRAINING INFORMATION ****")
  print("Classification Report and Confusion Matrix on Training")
  print(classification_report(y_train, y_train_check))
  print(confusion_matrix(y_train, y_train_check, labels=np.unique(y_train)))
  print("------------------------------")
  print("**** VALIDATION INFORMATION ****")
  y_pred_SVM = svm.predict(X_val)
  print("Classification Report and Confusion Matrix on Validation")
  print(classification_report(y_val, y_pred_SVM))
  print(confusion_matrix(y_val, y_pred_SVM, labels=np.unique(y_val)))
  pkl_filename = "SVM_pickle_model.pkl"
  with open(pkl_filename, 'wb') as file:
    pickle.dump(svm, file)
  return score_train_acc, score_val_acc

def XGBoost_model(X_train, y_train,X_val,y_val):
  XGB = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.4,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=3, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softmax', random_state=0,
              reg_alpha=1e-05, reg_lambda=1, scale_pos_weight=1, seed=2,
              silent=None, subsample=0.8, verbosity=1)
  XGB.fit(X_train, y_train)
  score_train_acc = XGB.score(X_train, y_train)
  print("Training Accuracy", score_train_acc)
  score_val_acc = XGB.score(X_val, y_val)
  print("Validation Accuracy", score_val_acc)  
  y_train_check = XGB.predict(X_train)
  print("**** TRAINING INFORMATION ****")
  print("Classification Report and Confusion Matrix on Training")
  print(classification_report(y_train, y_train_check))
  print(confusion_matrix(y_train, y_train_check, labels=np.unique(y_train)))
  print("------------------------------")
  print("**** VALIDATION INFORMATION ****")
  y_pred_XGB = XGB.predict(X_val)
  print("Classification Report and Confusion Matrix on Validation")
  print(classification_report(y_val, y_pred_XGB))
  print(confusion_matrix(y_val, y_pred_XGB, labels=np.unique(y_pred_XGB)))
  pkl_filename = "XGB_pickle_model.pkl"
  with open(pkl_filename, 'wb') as file:
    pickle.dump(XGB, file)
  return score_train_acc, score_val_acc

def KNN_model(X_train, y_train,X_val,y_val):
  knn = KNeighborsClassifier(n_neighbors=5,weights='uniform',algorithm='auto')
  knn.fit(X_train, y_train)
  score_train_acc = knn.score(X_train, y_train)
  print("Training Accuracy", score_train_acc) 
  score_val_acc = knn.score(X_val, y_val)
  print("Validation Accuracy", score_val_acc)
  y_train_check = knn.predict(X_train)
  print("**** TRAINING INFORMATION ****")
  print("Classification Report and Confusion Matrix on Training")
  print(classification_report(y_train, y_train_check))
  print(confusion_matrix(y_train, y_train_check, labels=np.unique(y_train)))
  print("------------------------------")
  print("**** VALIDATION INFORMATION ****")
  y_pred_knn = knn.predict(X_val) 
  print("Classification Report and Confusion Matrix on Validation")
  print(classification_report(y_val, y_pred_knn))
  print(confusion_matrix(y_val, y_pred_knn, labels=np.unique(y_pred_knn)))
  pkl_filename = "KNN_pickle_model.pkl"
  with open(pkl_filename, 'wb') as file:
    pickle.dump(knn, file)
  return score_train_acc, score_val_acc

def ANN(learning_rate):
  adam = Adam(lr = learning_rate)
  classifier = Sequential()
  classifier.add(Dense(units = 128, activation = 'relu', 
                       kernel_initializer = 'random_uniform', input_dim = 250 ))
  classifier.add(Dropout(0.4))
  classifier.add(Dense(units = 128, activation = 'relu', 
                       kernel_initializer = 'random_uniform'))
  classifier.add(Dropout(0.4))
  classifier.add(Dense(units = 5, activation = 'softmax', 
                       kernel_initializer = 'random_uniform'))
  classifier.compile(optimizer = adam, loss = 'sparse_categorical_crossentropy',
                     metrics = ['sparse_categorical_accuracy'])
  print(classifier.summary())
  return classifier

In [0]:
def train_val_model(models):
  print("The model user have provided for tarining:", models)
  df = pd.read_csv("train.csv") # Creating DataFrame
  print("DataFrame",df.shape) # Checking the dimension
  X = df.iloc[:,1:-1].values # Converting DataFrame to Numpy and ignoring "id"
  print("X",X.shape) # Checking the dimension
  y = df.iloc[:, df.shape[1]-1].values # Converting DataFrame to Numpy for Ground Truth
  print(y.shape) # Checking the dimension
  # Normalizing the inpu
  sc = StandardScaler() 
  X = sc.fit_transform(X) 
  learning_rate = 0.0001  # Used for Adam Optimizer
  iterr = 100 # Number of Epochs
  n_folds = 5 # Number of folds for Cross Validation
  bsize = 32  # Batch Size
  train_acc, val_acc = [], [] # Lists for storing accuracies on each fold

  # Cross validation
  kfold = KFold(n_splits=n_folds, shuffle=True, random_state=7)
  hist = []
  val_hist=[]
  acc=[]
  val_acc=[]
  for train, val in kfold.split(X, y):
    X_train, X_val = X[train], X[val] # Grabbing X_train, X_val for Models
    y_train, y_val = y[train], y[val] # Grabbiing y_train, y_val for Models
    if models == "ANN":
        ann_model = ANN(learning_rate)
        history = ann_model.fit(X_train, y_train,validation_data=(X_val,y_val), epochs=iterr, batch_size=bsize, verbose=1)
        #print(history.history.keys())
        hist.append(history.history['loss'])
        val_hist.append(history.history['val_loss'])
        acc.append(history.history['sparse_categorical_accuracy'])
        val_acc.append(history.history['val_sparse_categorical_accuracy'])
        plt.figure()
        plt.subplot(121)
        # For Plotting, X_axis: Number of Epochs, Y_axis = Loss
        plt.title("Trainig Loss")
        plt.plot(history.history['loss'])
        plt.subplot(122)
        plt.title("Validation Loss")
        plt.plot(history.history['val_loss'])
        plt.show()
        yield hist,val_hist,acc,val_acc

    elif models == "LR":
        lr_model = Logistic_Regression_model(X_train, y_train,X_val,y_val)
        train_acc.append(lr_model[0])
        val_acc.append(lr_model[1])
        #print("TA",train_acc)
        print("Training Accuracy is:", sum(train_acc)/n_folds)
        print("Validation Accuracy is:", sum(val_acc)/n_folds)

    elif models == "SVM":
        svm_model = SVM_model(X_train, y_train,X_val,y_val)
        train_acc.append(svm_model[0])
        val_acc.append(svm_model[1])
        #print("TA",train_acc)
        print("Train Accuracy is:", sum(train_acc)/n_folds)
        print("Validation Accuracy is:", sum(val_acc)/n_folds)

    elif models == "XGB":
        xgb_model = XGBoost_model(X_train, y_train,X_val,y_val)
        train_acc.append(xgb_model[0])
        val_acc.append(xgb_model[1])
        #print("TA",train_acc)
        print("Train Accuracy is:", sum(train_acc)/n_folds)
        print("Validation Accuracy is:", sum(val_acc)/n_folds)

    elif models == "KNN":
        knn_model = KNN_model(X_train, y_train,X_val,y_val)
        train_acc.append(knn_model[0])
        val_acc.append(knn_model[1])
        #print("TA",train_acc)
        print("Train Accuracy is:", sum(train_acc)/n_folds)
        print("Validation Accuracy is:", sum(val_acc)/n_folds)


**Information on Training Models**

-> I'm using KFold Crossvalidation and splitting the Trainig data into Training and Validation

-> I have used GridSearchCV for best params

-> Also, I have **Hypertuned** all the experimented models and observed accuracy, loss, precision, recall, f1_scores

-> After observing all the experimented models I decided to use one model for Prediction as it has the least overfiting amongst all

In [23]:
if __name__ == "__main__":
  model = "KNN" # Depends on User, User can select models to experiment from the given list ["LR","SVM","XGB","KNN","ANN"]
  if model == "ANN":
    try:
      hist,val_hist,acc,val_acc = train_val_model(models=model)
    except Exception as e:
      pass
  else:
    _ = train_val_model(models=model)
    for el in _: print(el)

The model user have provided for tarining: KNN
DataFrame (7200, 252)
X (7200, 250)
(7200,)
Training Accuracy 0.9673611111111111
Validation Accuracy 0.9326388888888889
**** TRAINING INFORMATION ****
Classification Report and Confusion Matrix on Training
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      1097
           1       0.94      0.99      0.96      1156
           2       0.97      0.98      0.97      1200
           3       0.99      0.93      0.96      1136
           4       0.99      0.96      0.97      1171

    accuracy                           0.97      5760
   macro avg       0.97      0.97      0.97      5760
weighted avg       0.97      0.97      0.97      5760

[[1082    5    4    3    3]
 [   8 1139    3    3    3]
 [   3   18 1173    3    3]
 [  14   37   20 1057    8]
 [  14   14   15    7 1121]]
------------------------------
**** VALIDATION INFORMATION ****
Classification Report and Confusion Matrix on Validat

**Finalizing Model**

**Results**

**Logistic Regression**

Training Accuracy: 0.7929166666666667 

Validation Accuracy is: 0.7455555555555555


**SVM**

Training Accuracy: 0.9930902777777778 

Validation Accuracy is: 0.9823611111111111

**XGB**

Training Accuracy: 0.1 

Validation Accuracy is: 0.8411

**KNN**

Training Accuracy: 0.9680208333333333 

Validation Accuracy is: 0.9330555555555555

**My Model**

Training Accuracy: 0.98834 

Validation Accuracy is: 0.96058

-> After considering various factors as mentioned previously I used K Nearest Neighbors for predicting the ground truth on Test data

In [15]:
# Finalised Model

# Training DataFrame
df = pd.read_csv("train.csv")
print("DataFrame",df.shape)
X = df.iloc[:,1:-1].values
print("X_shape",X.shape)
y = df.iloc[:, df.shape[1]-1].values

# Testing DataFrame
df_test = pd.read_csv("test.csv")

ret_X = df_test.iloc[:,0].values # This will be used for Prediction.csv
#print(ret_X)
print("df_test",df_test.shape)

X_test = df_test.iloc[:,1:].values
print("X_test",X_test.shape)

# Normalizing the Data
sc = StandardScaler()
X_test = sc.fit_transform(X_test)
X = sc.fit_transform(X)


DataFrame (7200, 252)
X_shape (7200, 250)
df_test (4800, 251)
X_test (4800, 250)


Use the following two consecutive blocks only if the trained model is not saved on the disck.

I faced this problem when I was working on the Jupyter Notebook.

Savig the model on Google Colab has no issues and you can directly dp prediction

In [0]:
# Only use when model is not being saved
"""
def KNN_model(X_train, y_train,X_val,y_val, X_test):
  knn = KNeighborsClassifier(n_neighbors=5,weights='uniform',algorithm='auto')
  knn.fit(X_train, y_train)
  score_train_acc = knn.score(X_train, y_train)
  print("Training Accuracy", score_train_acc)
  score_val_acc = knn.score(X_val, y_val)
  print("Validating Accuracy", score_val_acc)
  y_pred_test = knn.predict(X_test)
  return y_pred_test"""

In [38]:
"""
kfold = KFold(n_splits=5, shuffle=True, random_state=7)
for train, val in kfold.split(X, y):
    X_train, X_val = X[train], X[val]
    y_train, y_val = y[train], y[val]
    
res = KNN_model(X_train, y_train,X_val,y_val, X_test)
print(len(res)) # Sanity check to match test input data rows"""

Training Accuracy 0.9670138888888888
Validating Accuracy 0.9326388888888889
4800


In [39]:
#np.unique(res).shape[0] # Checking number of unique predicted class

5

In [0]:
#ret = np.column_stack((ret_X, res)) # Combining the "id" column and "predicted class" for Prediction.csv

In [42]:
print(ret)

[[ 7200     4]
 [ 7201     2]
 [ 7202     2]
 ...
 [11997     4]
 [11998     4]
 [11999     4]]


In [0]:
# Saving the Prediction.csv  
#pd.DataFrame(ret).to_csv("prediction.csv", sep = "|", header = ["id", "Predictions"], index = False)

In [0]:
# Load from saved model from file
with open("KNN_pickle_model.pkl", 'rb') as file:
    pickle_model = pickle.load(file)

In [0]:
y_predictions = pickle_model.predict(X_test)

In [26]:
np.unique(y_predictions).shape[0]

5

In [0]:
ret = np.column_stack((ret_X, y_predictions))

In [28]:
print(ret)
print(len(ret))

[[ 7200     4]
 [ 7201     2]
 [ 7202     2]
 ...
 [11997     4]
 [11998     4]
 [11999     4]]
4800


In [ ]:
# Saving the Prediction.csv  
pd.DataFrame(ret).to_csv("Predictions.csv", sep = "|", header = ["id", "label"], index = False)